<a href="https://colab.research.google.com/github/amankiitg/LLM_Prod/blob/main/VLLM_Phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Make sure you have vLLM installed
!pip install vllm transformers>=4.40.1 accelerate>=0.27.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.


In [2]:
# 1. Define the model name
model_name = "microsoft/Phi-3-mini-4k-instruct"

In [3]:
# 2. Load the tokenizer from Hugging Face Transformers
# vLLM uses the tokenizer to process the prompt correctly.
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


INFO 08-02 15:32:48 [__init__.py:235] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
# 3. Create the prompt using the model's chat template
# This part remains the same, as the model expects a specific format.
messages = [
    {"role": "user", "content": "Tell me a short summary of the 2013 movie HER"}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)


In [ ]:
# 4. Initialize the vLLM engine
# vLLM is an inference engine that loads the model and runs it efficiently.
# We explicitly set parameters to guide device detection and memory allocation.
# `enforce_eager=True` can help with compatibility in Colab environments.
# We still need `trust_remote_code=True` for the Phi-3 model architecture.

#tensor_parallelsim: splits large models across multiple GPUs
#enforce_eager: prevents going into graph compilation mode. Can be slower for inference, but avoids errors

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    tensor_parallel_size=1,
    enforce_eager=True, # Added for Colab compatibility
    dtype="auto",
    gpu_memory_utilization=0.9
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


WARNING 08-02 03:58:44 [config.py:3392] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 08-02 03:58:44 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-02 03:58:44 [config.py:1604] Using max model len 4096
WARNING 08-02 03:58:44 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 08-02 03:58:44 [cuda.py:103] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 08-02 03:58:44 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='microsoft/Phi-3-mini-4k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-mini-4k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, downl

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

INFO 08-02 04:01:29 [weight_utils.py:312] Time spent downloading weights for microsoft/Phi-3-mini-4k-instruct: 161.810306 seconds


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-02 04:01:58 [default_loader.py:262] Loading weights took 28.94 seconds
INFO 08-02 04:01:59 [model_runner.py:1115] Model loading took 7.1184 GiB and 191.442279 seconds
INFO 08-02 04:02:02 [worker.py:295] Memory profiling takes 2.59 seconds
INFO 08-02 04:02:02 [worker.py:295] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 08-02 04:02:02 [worker.py:295] model weights take 7.12GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.32GiB; the rest of the memory reserved for KV Cache is 5.80GiB.
INFO 08-02 04:02:03 [executor_base.py:113] # cuda blocks: 989, # CPU blocks: 682
INFO 08-02 04:02:03 [executor_base.py:118] Maximum concurrency for 4096 tokens per request: 3.86x
INFO 08-02 04:02:06 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 6.40 seconds


In [ ]:
sampling_params = SamplingParams(
    max_tokens=500,  # Corresponds to `max_new_tokens`
    temperature=0    # A temperature of 0 means greedy decoding (no randomness)
)

In [ ]:
outputs = llm.generate([prompt], sampling_params)



Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
# 7. Print the generated text
# The result is a list of `RequestOutput` objects. We access the first one.
# The `outputs` attribute of a `RequestOutput` is a list of `CompletionOutput` objects.
# We get the text from the first completion.
print(outputs[0].outputs[0].text)

 "Her" is a romantic science fiction film directed by Spike Jonze, released in 2013. The story follows Theodore Twombly, a lonely and unfulfilled man who develops a relationship with Samantha, an artificially intelligent operating system. Samantha, who has her own personality and consciousness, becomes Theodore's confidante and love interest. As their relationship deepens, Theodore begins to question the nature of love and connection. The film explores themes of human emotion, technology, and the complexities of relationships in a futuristic setting.
